In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

# -----------------------------
# 1. Adım: Verileri Yükleme
# -----------------------------
train_file = r'/content/drive/My Drive/akış-diyagramı-örnekleri-verileri.xlsx'  # Eğitim veri dosyası
valid_file = r'/content/drive/My Drive/akışlar-test-verileri.xlsx'  # Test veri dosyası

df_train = pd.read_excel(train_file)
df_valid = pd.read_excel(valid_file)

# -----------------------------
# 2. BERT ile Adım Türü Tahmini
# -----------------------------

# Dataset sınıfı
class BertDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# BERT Modeli ve Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased", token='<your_actual_token>')
bert_model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", num_labels=8, token='<your_actual_token>')

# Eğitim verileri hazırlama
bert_train_X = [f"Adım No: {row['Adım No']}, Adım Açıklaması: {row['Adım Açıklaması']}" for _, row in df_train.iterrows()]
bert_train_y = df_train['Adım Türü Label'].values
bert_valid_X = [f"Adım No: {row['Adım No']}, Adım Açıklaması: {row['Adım Açıklaması']}" for _, row in df_valid.iterrows()]
bert_valid_y = df_valid['Adım Türü Label'].values

train_dataset = BertDataset(bert_train_X, bert_train_y, bert_tokenizer)
valid_dataset = BertDataset(bert_valid_X, bert_valid_y, bert_tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

# Eğitim döngüsü
optimizer = torch.optim.AdamW(bert_model.parameters(), lr=5e-5)
bert_model.train()
train_losses = []
valid_losses = []

for epoch in range(8):  #----------------------------------------------------------------------------------------- 3 Epoch
    epoch_train_loss = 0
    bert_model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = bert_model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['labels']
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    train_losses.append(epoch_train_loss / len(train_loader))

    # Doğrulama sırasında loss hesaplama
    epoch_valid_loss = 0
    bert_model.eval()
    with torch.no_grad():
        for batch in valid_loader:
            outputs = bert_model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )
            loss = outputs.loss
            epoch_valid_loss += loss.item()
    valid_losses.append(epoch_valid_loss / len(valid_loader))

    print(f"Epoch {epoch + 1} | Train Loss: {train_losses[-1]:.4f} | Validation Loss: {valid_losses[-1]:.4f}")

# BERT Doğrulama Performansı
bert_predictions = []
bert_model.eval()
with torch.no_grad():
    for text in bert_valid_X:
        encoding = bert_tokenizer(
            text,
            max_length=128,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        output = bert_model(
            input_ids=encoding['input_ids'],
            attention_mask=encoding['attention_mask']
        )
        preds = torch.argmax(output.logits, dim=1).item()
        bert_predictions.append(preds)

# Performans Metrikleri
print("BERT Model Accuracy:", accuracy_score(bert_valid_y, bert_predictions))
print("BERT Classification Report:")
print(classification_report(bert_valid_y, bert_predictions))

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences

# -----------------------------
# 2. Adım: Veriyi Hazırlama
# -----------------------------
# Adım açıklamaları ve koşul etiketini çıkarıyoruz
df_train['Adım Açıklaması'] = df_train['Adım Açıklaması'].astype(str)
df_valid['Adım Açıklaması'] = df_valid['Adım Açıklaması'].astype(str)

# Koşul türü ile ilişkili olan adım sıralarını tahmin etmeyi amaçlıyoruz
# Koşul (evet/hayır) verisini kullanacağız (Koşul Label)

# Adım türüne göre koşul (adım türü koşul olanlar)
df_train_condition = df_train[df_train['Adım Türü'] == 'Koşul']
df_valid_condition = df_valid[df_valid['Adım Türü'] == 'Koşul']

# Adım türüne göre sırasıyla adımları tahmin edeceğiz (LSTM kullanarak)
# Adım sırasını ve koşul sonrasındaki adımları tahmin etme

# -----------------------------
# 3. LSTM Modeli İçin Veri Hazırlığı
# -----------------------------
# Adım açıklamalarını sayısal verilere dönüştürmek için tokenizer kullanıyoruz
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['Adım Açıklaması'])
X_train = tokenizer.texts_to_sequences(df_train['Adım Açıklaması'])
X_valid = tokenizer.texts_to_sequences(df_valid['Adım Açıklaması'])

# Veriyi sabit uzunlukta tutmak için pad_sequence kullanıyoruz
max_sequence_length = 100  # Bu değeri verinin uzunluğuna göre ayarlayabilirsiniz
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_valid = pad_sequences(X_valid, maxlen=max_sequence_length)

# Koşul etiketlerini alıyoruz (Koşul => Evet ve Hayır durumu)
y_train_condition = df_train_condition['Adım Türü Label'].values
y_valid_condition = df_valid_condition['Adım Türü Label'].values

# Adım sıralarını alıyoruz (Koşul dışındaki her adım sırası)
y_train_step = df_train['Adım No'].values
y_valid_step = df_valid['Adım No'].values

# -----------------------------
# 4. LSTM Modeli: Adım Sırası ve Koşul Durumu
# -----------------------------
# LSTM Modeli (Adım Sırası ve Koşul Durumu için)
lstm_model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length),
    LSTM(128, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Bu çıkış, adım sırasını tahmin etmek için
])

lstm_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Adım sırasını tahmin etme (sürekli değişken olduğu için 'linear' aktivasyon)
lstm_model.fit(X_train, y_train_step, epochs=1809, batch_size=512, validation_data=(X_valid, y_valid_step)) #----------------------epoch

# -----------------------------
# 5. Koşul Durumu için LSTM Modeli
# -----------------------------
# Koşul adımlarında (Evet/Hayır) tahmin yapmak için farklı bir model
condition_model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length),
    LSTM(128, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Koşul doğru/yanlış tahmin etmek için
])

condition_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Koşul etiketini tahmin etme (evet/hayır)
# Filter X_train to include only the rows corresponding to 'Koşul' steps
X_train_condition = X_train[df_train['Adım Türü'] == 'Koşul']
X_valid_condition = X_valid[df_valid['Adım Türü'] == 'Koşul']

# Now fit the model with the filtered data
condition_model.fit(
    X_train_condition,
    y_train_condition,
    epochs=235,                       #---------------------------------------------------------------------------------epoch
    batch_size=16,
    validation_data=(X_valid_condition, y_valid_condition)
)
# -----------------------------
# 6. Sonuçları Değerlendirme
# -----------------------------
# Adım sırasını tahmin etme (Adım No) için
step_predictions = lstm_model.predict(X_valid)
step_predictions = np.round(step_predictions)

# Koşul durumunu tahmin etme (Koşul Label) için
condition_predictions = condition_model.predict(X_valid)
condition_predictions = (condition_predictions > 0.5).astype(int)

condition_predictions = condition_model.predict(X_valid_condition)
condition_predictions = (condition_predictions > 0.5).astype(int)

# Adım sırası ve koşul durumunun doğruluğunu değerlendirme
print("Adım Sırası Doğruluk Skoru:", accuracy_score(y_valid_step, step_predictions))
print("Koşul Durumu Doğruluk Skoru:", accuracy_score(y_valid_condition, condition_predictions))
print("Adım Sırası Sınıflandırma Raporu:")
print(classification_report(y_valid_step, step_predictions))

print("Koşul Durumu Sınıflandırma Raporu:")
print(classification_report(y_valid_condition, condition_predictions))